In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.applications import MobileNetV2,MobileNetV3Small,EfficientNetV2B0
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import pandas as pd
import numpy as np
import datetime
import os
import tensorflow.keras.backend as K
import datetime
from sklearn.metrics import roc_curve, auc
import cv2
import random
from tensorflow.keras.mixed_precision import set_global_policy, Policy

# # Enable memory growth for both T4 GPUs
# gpus = tf.config.list_physical_devices('GPU')
# print("Available GPUs:", [gpu.name for gpu in gpus])  # Should list two T4 GPUs
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)



# Initialize MirroredStrategy
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
print("Number of GPUs in strategy:", strategy.num_replicas_in_sync)  # Should print 2

Number of GPUs in strategy: 2


In [16]:
BATCH_SIZE = 64
IMG_SIZE = (112, 112)
DATA_DIR = "/kaggle/input/11-785-fall-20-homework-2-part-2"
TRAIN_DIR = f"{DATA_DIR}/classification_data/train_data"
VERIFICATION_FILE = os.path.join(DATA_DIR, "verification_pairs_val.txt")
NUM_PAIRS_PER_PERSON = 25


In [17]:

def load_and_preprocess_image(image_path, img_size=IMG_SIZE):
    if isinstance(image_path, str):
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not load image: {image_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        image = image_path
    image = cv2.resize(image, img_size)
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    return image

def create_pairs_from_verification_file(verification_file_path=VERIFICATION_FILE, base_dir=DATA_DIR):
    pairs = []
    with open(verification_file_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 3:
                img1_path = os.path.join(base_dir, parts[0])
                img2_path = os.path.join(base_dir, parts[1])
                label = int(parts[2])
                pairs.append((img1_path, img2_path, label))
    return pairs

def create_pairs_from_classification_data(classification_dir=TRAIN_DIR, num_pairs_per_person=NUM_PAIRS_PER_PERSON):
    pairs = []
    person_dirs = [d for d in os.listdir(classification_dir) 
                  if os.path.isdir(os.path.join(classification_dir, d))]
    
    for person_dir in person_dirs:
        person_path = os.path.join(classification_dir, person_dir)
        images = [f for f in os.listdir(person_path) 
                 if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        if len(images) >= 2:
            for _ in range(num_pairs_per_person):
                img1, img2 = random.sample(images, 2)
                img1_path = os.path.join(person_path, img1)
                img2_path = os.path.join(person_path, img2)
                pairs.append((img1_path, img2_path, 1))
    
    num_negative_pairs = len(pairs)
    for _ in range(num_negative_pairs):
        person1, person2 = random.sample(person_dirs, 2)
        person1_path = os.path.join(classification_dir, person1)
        person2_path = os.path.join(classification_dir, person2)
        images1 = [f for f in os.listdir(person1_path) 
                  if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        images2 = [f for f in os.listdir(person2_path) 
                  if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        if images1 and images2:
            img1 = random.choice(images1)
            img2 = random.choice(images2)
            img1_path = os.path.join(person1_path, img1)
            img2_path = os.path.join(person2_path, img2)
            pairs.append((img1_path, img2_path, 0))
    
    random.shuffle(pairs)
    return pairs

def create_data_augmentation_layer():
    return keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ])

def prepare_dataset(pairs, batch_size=BATCH_SIZE, shuffle=True, augment=True):
    def load_pair(img1_path, img2_path, label):
        img1 = load_and_preprocess_image(img1_path.numpy().decode("utf-8"))
        img2 = load_and_preprocess_image(img2_path.numpy().decode("utf-8"))
        
        # Convert to tensors with proper shape
        img1 = tf.convert_to_tensor(img1, dtype=tf.float32)
        img2 = tf.convert_to_tensor(img2, dtype=tf.float32)
        
        return (img1, img2, label)
    
    img1_paths = [pair[0] for pair in pairs]
    img2_paths = [pair[1] for pair in pairs]
    labels = [pair[2] for pair in pairs]
    
    dataset = tf.data.Dataset.from_tensor_slices((img1_paths, img2_paths, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(pairs))
    
    dataset = dataset.map(
        lambda p1, p2, l: tf.py_function(
            load_pair, [p1, p2, l], [tf.float32, tf.float32, tf.int32]
        ),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    dataset = dataset.map(
        lambda img1, img2, label: (
            (tf.ensure_shape(img1, [*IMG_SIZE, 3]),
             tf.ensure_shape(img2, [*IMG_SIZE, 3])),
            tf.ensure_shape(label, [])
        )
    )
    
    dataset = dataset.batch(batch_size)
    
    # Apply augmentation AFTER batching
    if augment:
        augment_layer = create_data_augmentation_layer()
        def augment_batch(batch_images, batch_labels):
            img1_batch, img2_batch = batch_images
            img1_batch = augment_layer(img1_batch)
            img2_batch = augment_layer(img2_batch)
            return (img1_batch, img2_batch), batch_labels
        
        dataset = dataset.map(augment_batch, num_parallel_calls=tf.data.AUTOTUNE)
    
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

def load_verification_data(verification_file_path=VERIFICATION_FILE, base_dir=DATA_DIR):
    pairs = create_pairs_from_verification_file(verification_file_path, base_dir)
    images1, images2, labels = [], [], []
    
    for img1_path, img2_path, label in pairs:
        try:
            img1 = load_and_preprocess_image(img1_path)
            img2 = load_and_preprocess_image(img2_path)
            images1.append(img1)
            images2.append(img2)
            labels.append(label)
        except Exception as e:
            print(f"Error loading pair {img1_path}, {img2_path}: {e}")
            continue
    
    return np.array(images1), np.array(images2), np.array(labels)

In [18]:
# Prepare training dataset (from classification folder)
train_pairs = create_pairs_from_classification_data()
train_dataset = prepare_dataset(train_pairs, batch_size=BATCH_SIZE, augment=True)

# Prepare validation dataset (from verification file)
val_images1, val_images2, val_labels = load_verification_data()
val_pairs = create_pairs_from_verification_file()
val_dataset = prepare_dataset(val_pairs, batch_size=BATCH_SIZE, augment=False)

In [19]:
# def contrastive_loss(y_true, y_pred, margin=MARGIN):
#     y_true = tf.cast(y_true, tf.float32)
#     y_pred = tf.cast(y_pred, tf.float32)
#     loss = y_true * tf.square(y_pred) + (1 - y_true) * tf.square(tf.maximum(margin - y_pred, 0))
#     return tf.reduce_mean(loss)

def euclidean_distance(embeddings):
    embedding1, embedding2 = embeddings
    return tf.sqrt(tf.reduce_sum(tf.square(embedding1 - embedding2), axis=1, keepdims=True))

def cosine_distance(embeddings):
    embedding1, embedding2 = embeddings
    cosine_sim = tf.reduce_sum(embedding1 * embedding2, axis=1, keepdims=True)
    return 1 - cosine_sim

In [20]:
EMBEDDING_DIM = 128
FREEZE_BACKBONE = True
LEARNING_RATE = 0.0001
def create_embedding_network(embedding_dim=EMBEDDING_DIM, freeze_backbone=FREEZE_BACKBONE):
    
    base_model = EfficientNetV2B0(
        weights="imagenet", include_top=False, input_shape=(*IMG_SIZE, 3)
    )
    # if freeze_backbone:
    #     base_model.trainable = False
    base_model.trainable = True
        

    inputs = keras.Input(shape=(*IMG_SIZE, 3))
    x = keras.applications.efficientnet_v2.preprocess_input(inputs)
    
    x = base_model(x)
    x = layers.GlobalAveragePooling2D()(x)
    
    x = layers.Dense(512, name="embedding_dense1")(x)
    x = layers.BatchNormalization(name="embedding_bn1")(x)
    x = layers.Activation('relu', name = "activation_bn1")(x)
    x = layers.Dropout(0.4, name="embedding_dropout1")(x)
    
    # x = layers.Dense(256, name="embedding_dense2")(x)
    # x = layers.BatchNormalization(name="embedding_bn2")(x)
    # x = layers.Activation('relu', name = "activation_bn2")(x)
    # x = layers.Dropout(0.3, name="embedding_dropout2")(x)
    


    # Final embedding layer with proper dimensionality
    embeddings = layers.Dense(embedding_dim, name="final_embeddings")(x)
    # L2 normalize embeddings for better cosine similarity computation
    embeddings = layers.Lambda(lambda x: K.l2_normalize(x, axis=1), name="l2_normalize",output_shape=(embedding_dim,))(embeddings)
    
    return Model(inputs, embeddings)

def unfreeze_backbone(model):
    backbone = model.layers[2]  # Base model layer
    backbone.trainable = True
    return model



def create_siamese_model(input_shape=(*IMG_SIZE, 3), embedding_dim=EMBEDDING_DIM, 
                      
                        freeze_backbone=FREEZE_BACKBONE, learning_rate=LEARNING_RATE):
    input1 = keras.Input(shape=input_shape, name="image1")
    input2 = keras.Input(shape=input_shape, name="image2")
    
    embedding_network = create_embedding_network(embedding_dim, freeze_backbone)
    
    embedding1 = embedding_network(input1)
    embedding2 = embedding_network(input2)
    
    def cosine_similarity(vectors):
        x, y = vectors
        return K.sum(x * y, axis=1, keepdims=True)
    
    similarity = layers.Lambda(cosine_similarity, name="cosine_similarity",output_shape=(1,))([embedding1, embedding2])
    
    # Additional processing layers to learn optimal decision boundary
    x = layers.Dense(64, activation='relu', name="similarity_dense1")(similarity)
    x = layers.Dropout(0.2, name="similarity_dropout")(x)
    # x = layers.Dense(32, activation='relu', name="similarity_dense2")(x)
    
    # Final binary classification layer
    output = layers.Dense(1, activation='sigmoid', name="binary_output")(x)
    
    # Build and compile model
    model = keras.Model(inputs=[input1, input2], outputs=output)
    
    model = keras.Model(inputs=[input1, input2], outputs=output)
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate if freeze_backbone else learning_rate/10)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=["accuracy"])
    return model

In [21]:

# Build model
model = create_siamese_model()
model.summary()
# model = load_model("/kaggle/working/best_Siamese_model_20250719_123439.keras")
# model.summary()

optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=["accuracy"])
# Define callbacks

checkpoint = callbacks.ModelCheckpoint(
    f"best_Siamese_model_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.keras"
    , monitor="val_accuracy", save_best_only=True, mode="auto", verbose=1
)
early_stopping = callbacks.EarlyStopping(
    monitor="val_accuracy", patience=3, restore_best_weights=True, verbose=1
)
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor="val_accuracy", factor=0.2, patience=2, min_lr=1e-6, verbose=1
)
# Train model (Phase 1: Frozen backbone)
history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=100,
        verbose=1,
        callbacks=[checkpoint, early_stopping, reduce_lr],
        initial_epoch = 0
    
    )

# Save final model
model.save(f"final_Siamese_model_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.keras")

# Save training history
pd.DataFrame(history.history).to_csv(f"history_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image1 (InputLayer) │ (None, 112, 112,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ image2 (InputLayer) │ (None, 112, 112,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_7        │ (None, 128)       │  6,642,896 │ image1[0][0],     │
│ (Functional)        │                   │            │ image2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cosine_similarity   │ (None, 1)         │          0 │ functional_7[0][… │
│ (Lambda)            │                   │            │ functional_7[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ similarity_dense1   │ (None, 64)        │        128 │ cosine_similarit… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ similarity_dropout  │ (None, 64)        │          0 │ similarity_dense… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ binary_output       │ (None, 1)         │         65 │ similarity_dropo… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,643,089 (25.34 MB)

 Trainable params: 6,581,457 (25.11 MB)

 Non-trainable params: 61,632 (240.75 KB)

Epoch 1/100


I0000 00:00:1753178905.127565      93 service.cc:148] XLA service 0x7fe7e40020e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753178905.128439      93 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753178915.154561      93 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1753178930.390102      93 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1753178930.573647      93 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1753178931.081389      93 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1753178931.2881

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step - accuracy: 0.6536 - loss: 0.6155
Epoch 1: val_accuracy improved from -inf to 0.76865, saving model to best_Siamese_model_20250722_100651.keras
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1434s 409ms/step - accuracy: 0.6536 - loss: 0.6155 - val_accuracy: 0.7687 - val_loss: 0.4848 - learning_rate: 1.0000e-04
Epoch 2/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step - accuracy: 0.8141 - loss: 0.4200
Epoch 2: val_accuracy improved from 0.76865 to 0.79671, saving model to best_Siamese_model_20250722_100651.keras
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 881s 282ms/step - accuracy: 0.8141 - loss: 0.4200 - val_accuracy: 0.7967 - val_loss: 0.4442 - learning_rate: 1.0000e-04
Epoch 3/100
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step - accuracy: 0.8402 - loss: 0.3705
Epoch 3: val_accuracy improved from 0.79671 to 0.80625, saving model to best_Siamese_model_20250722_100651.keras
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 890s 285ms/step - accuracy: 0.8402 - loss: 0.3705 - val_accuracy: 0.8

In [22]:
# Optional: Fine-tune (Phase 2)
# model = unfreeze_backbone(model)
# model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='binary_crossentropy')
# model.fit(
#         train_dataset,
#         validation_data=val_dataset,
#         epochs=100,
#         initial_epoch = 10,
#         verbose=1,
#         callbacks=[checkpoint, early_stopping, reduce_lr],
    
#     )